# Generate list of Extension in CSV

In [47]:
import pandas as pd
import subprocess
import requests
from lxml import html

In [ ]:
# Run the command to list VS Code extensions
code_cmd = r'C:\Users\parki\AppData\Local\Programs\Microsoft VS Code\bin\code.cmd'
result = subprocess.run([code_cmd, '--list-extensions'], capture_output=True, text=True)
xpath = '//*[@id="section-banner"]/div/table/tbody/tr/td[2]/div/h1/span'  # Replace with your specific XPath
# Extract the list of extensions from the result
extensions = result.stdout.splitlines()

# URL generation
generated_url= ['https://marketplace.visualstudio.com/items?itemName=' + ext for ext in extensions]

# Function to extract specific XPath
def extract_xpath(url, xpath):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        tree = html.fromstring(response.content)
        content = tree.xpath(xpath)[0].text
        return content
    except (requests.exceptions.RequestException, IndexError):
        return None

# get name from xpath
extension_names = [extract_xpath(u, xpath) for u in generated_url]

# Create a new DataFrame with the required columns
df = pd.DataFrame({
    'extension name': extension_names,
    'identifier': extensions,
    'url': generated_url
})

# Save the new DataFrame to a CSV file
df.to_csv('extensions.csv', index=False)
with open('extensions.md', 'w') as f:
    f.write("# My VS Code Extensions\n\n")
    for index, row in df.iterrows():
        f.write(f"[{row['extension name']}]({row['url']})\n")
print(df)